In [1]:
import awkward as ak
import numpy as np
import os, json, time, uproot, yaml, correctionlib

from coffea import processor, lumi_tools
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.nanoaod import FatJetArray
from coffea.analysis_tools import PackedSelection
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory, CorrectedMETFactory

import sys
#sys.path.append("..")
from src.processors.Processor import Processor
from src.processors.TriggerProcessor import TriggerProcessor
from src.processors.GenMatch import GenMatch

#import importlib
#importlib.reload(Processor)

/data/bond/fudawei/anaconda3/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [5]:
uproot.open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource
_events = NanoEventsFactory.from_root(
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/GJets/GJets_HT-600ToInf/209.root': 'Events'},
    {'~/ZpToHG_M-1000.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/TTJets/TTJets/2339.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/ZpToHG/ZpToHGamma_M1000/14.root': 'Events'},
    schemaclass=NanoAODSchema, delayed=False,
).events()
p=Processor(outdir='./test/', mode='mc_2018_ZpToHG', param_dir='./src/parameters/')
c=p.process(_events)

/data/bond/fudawei/anaconda3/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/data/bond/fudawei/anaconda3/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


FieldNotFoundError: no field 'inclParTMDV2_probQCDb' in record with 79 fields

In [17]:
AK8jet = _events.FatJet

extract = extractor()
uncertainty_dir = os.path.join('../src/parameters', 'uncertainty', '2018')
for f in os.listdir(uncertainty_dir):
    if 'AK8' not in f or 'UncertaintySources' in f:
        continue
    extract.add_weight_sets([f'* * {os.path.join(uncertainty_dir, f)}'])
extract.finalize()
evaluator = extract.make_evaluator()

jec_names = dir(evaluator)
jec_inputs = {name: evaluator[name] for name in jec_names}
jec_stack = JECStack(jec_inputs)
name_map = jec_stack.blank_name_map
name_map['JetPt'] = 'pt'
name_map['JetMass'] = 'mass'
name_map['JetEta'] = 'eta'
name_map['JetPhi'] = 'phi'
name_map['JetA'] = 'area'
name_map['ptGenJet'] = 'pt_gen'
name_map['ptRaw'] = 'pt_raw'
name_map['massRaw'] = 'mass_raw'
name_map['Rho'] = 'PU_rho'

AK8jet['pt_original'] = AK8jet.pt
AK8jet['mass_original'] = AK8jet.mass
AK8jet['is_real'] = (~np.isnan(ak.fill_none(AK8jet.matched_gen.pt, np.nan)))*1
AK8jet['pt_raw'] = (1 - AK8jet.rawFactor) * AK8jet.pt
AK8jet['mass_raw'] = (1 - AK8jet.rawFactor) * AK8jet.mass
AK8jet['pt_gen'] = ak.values_astype(ak.fill_none(AK8jet.matched_gen.pt, 0), np.float32)
AK8jet['PU_rho'] = ak.broadcast_arrays(_events.fixedGridRhoFastjetAll, AK8jet)[0]

corrected_AK8jet = CorrectedJetsFactory(name_map, jec_stack).build(AK8jet).compute()
AK8jet['pt_nominal'] = corrected_AK8jet.pt
AK8jet['mass_nominal'] = corrected_AK8jet.mass

for i in corrected_AK8jet.fields:
    if i.startswith("JES") or i.startswith("JER"):
        AK8jet[f"pt_{i[:3]}_up"] = corrected_AK8jet[i].up.pt
        AK8jet[f"pt_{i[:3]}_down"] = corrected_AK8jet[i].down.pt
        AK8jet[f"mass_{i[:3]}_up"] = corrected_AK8jet[i].up.mass
        AK8jet[f"mass_{i[:3]}_down"] = corrected_AK8jet[i].down.mass

In [36]:
a=AK8jet['pt_JES_up']
b=AK8jet['pt']
c=AK8jet['pt_JES_down']
d=AK8jet['pt_raw']

In [37]:
ak.mean(a[ak.num(a)>0][:,0])

429.6364577504197

In [38]:
ak.mean(b[ak.num(b)>0][:,0])

446.37627308337994

In [39]:
ak.mean(c[ak.num(c)>0][:,0])

422.26382204812535

In [40]:
ak.mean(d[ak.num(d)>0][:,0])

405.4403749300504

In [41]:
corrected_AK8jet.pt

<Array [[], [300, 171], ..., [...], [403, 367]] type='150000 * var * float32'>

In [42]:
AK8jet.pt_nominal

<Array [[], [300, 171], ..., [...], [403, 367]] type='150000 * var * float32'>

In [43]:
AK8jet.pt

<Array [[], [318, 200], ..., [...], [440, 403]] type='150000 * var * float3...'>